<a href="https://colab.research.google.com/github/RCortez25/TinyTorch/blob/main/0.%20Foundation/01_tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
#| default_exp core.tensor
#| export

import numpy as np

# Constants for memory calculations
BYTES_PER_FLOAT32 = 4  # Standard float32 size in bytes
KB_TO_BYTES = 1024  # Kilobytes to bytes conversion
MB_TO_BYTES = 1024 * 1024  # Megabytes to bytes conversion

# Tensor class

In [160]:
#| export
class Tensor:
    """Educational tensor - the foundation of machine learning computation.

    This class provides the core data structure for all ML operations:
    - data: The actual numerical values (NumPy array)
    - shape: Dimensions of the tensor
    - size: Total number of elements
    - dtype: Data type (float32)

    All arithmetic, matrix, and shape operations are built on this foundation.
    """
    def __init__(self, data):
        """Create a new tensor from data.

        TODO: Initialize a Tensor by wrapping data in a NumPy array and setting attributes.

        APPROACH:
        1. Convert data to NumPy array with dtype=float32
        2. Store the array as self.data
        3. Set self.shape from the array's shape
        4. Set self.size from the array's size
        5. Set self.dtype from the array's dtype

        EXAMPLE:
        >>> t = Tensor([1, 2, 3])
        >>> print(t.shape)
        (3,)
        >>> print(t.size)
        3

        HINT: Use np.array(data, dtype=np.float32) to convert data to NumPy array
        """
        self.data = np.array(data, dtype=np.float32)
        self.shape = self.data.shape
        self.size = self.data.size
        self.dtype = self.data.dtype

    def __repr__(self):
        "String representation for debugging"
        return f"Tensor(data={self.data}, shape={self.shape})"

    def __str__(self):
        "String representation"
        return f"Tensor({self.data})"

    def numpy(self):
        "Return the NumPy array"
        return self.data

    def memory_footprint(self):
        """Calculate exact memory usage in bytes.

        Systems Concept: Understanding memory footprint is fundamental to ML systems.
        Before running any operation, engineers should know how much memory it requires.

        Returns:
            int: Memory usage in bytes (e.g., 1000x1000 float32 = 4MB)
        """
        return self.data.nbytes

    def __add__(self, other):
        """Add two tensors element-wise with broadcasting support.

        TODO: Implement element-wise addition that works with both Tensors and scalars.

        APPROACH:
        1. Check if other is a Tensor (use isinstance)
        2. If Tensor: add self.data + other.data
        3. If scalar: add self.data + other (broadcasting)
        4. Wrap result in new Tensor

        EXAMPLE:
        >>> a = Tensor([1, 2, 3])
        >>> b = Tensor([4, 5, 6])
        >>> c = a + b
        >>> print(c.data)
        [5. 7. 9.]

        HINT: NumPy's + operator handles broadcasting automatically
        """
        if isinstance(other, Tensor):
            return Tensor(self.data + other.data)
        return self.data + other

    def __sub__(self, other):
        """Subtract two tensors element-wise.

        TODO: Implement element-wise subtraction (same pattern as __add__).

        APPROACH:
        1. Check if other is a Tensor
        2. If Tensor: subtract self.data - other.data
        3. If scalar: subtract self.data - other
        4. Return new Tensor with result

        EXAMPLE:
        >>> a = Tensor([5, 7, 9])
        >>> b = Tensor([1, 2, 3])
        >>> c = a - b
        >>> print(c.data)
        [4. 5. 6.]

        HINT: Follow the same pattern as __add__ but with subtraction
        """

        if isinstance(other, Tensor):
            return Tensor(self.data - other.data)
        return self.data - other

    def __mul__(self, other):
        """Multiply two tensors element-wise (NOT matrix multiplication).

        TODO: Implement element-wise multiplication (same pattern as __add__).

        APPROACH:
        1. Check if other is a Tensor
        2. If Tensor: multiply self.data * other.data
        3. If scalar: multiply self.data * other
        4. Return new Tensor with result

        EXAMPLE:
        >>> a = Tensor([1, 2, 3])
        >>> b = Tensor([4, 5, 6])
        >>> c = a * b
        >>> print(c.data)
        [ 4. 10. 18.]

        HINT: Element-wise multiplication is *, not matrix multiplication (@)
        """
        if isinstance(other, Tensor):
            return Tensor(self.data * other.data)
        return self.data * other

    def __truediv__(self, other):
        """Divide two tensors element-wise.

        TODO: Implement element-wise division (same pattern as __add__).

        APPROACH:
        1. Check if other is a Tensor
        2. If Tensor: divide self.data / other.data
        3. If scalar: divide self.data / other
        4. Return new Tensor with result

        EXAMPLE:
        >>> a = Tensor([4, 6, 8])
        >>> b = Tensor([2, 2, 2])
        >>> c = a / b
        >>> print(c.data)
        [2. 3. 4.]

        HINT: Division creates float results automatically due to float32 dtype
        """
        if isinstance(other, Tensor):
            return Tensor(self.data / other.data)
        return Tensor(self.data / other)

    def matmul(self, other):
        """Matrix multiplication of two tensors.

        TODO: Implement matrix multiplication with shape validation.

        APPROACH:
        1. Validate other is a Tensor (raise TypeError if not)
        2. Check for scalar cases (0D tensors) - use element-wise multiply
        3. For 2D+ matrices: validate inner dimensions match (shape[-1] == shape[-2])
        4. For 2D matrices: use explicit nested loops (educational)
        5. For batched (3D+): use np.matmul for correctness
        6. Return result wrapped in Tensor

        EXAMPLE:
        >>> a = Tensor([[1, 2], [3, 4]])  # 2×2
        >>> b = Tensor([[5, 6], [7, 8]])  # 2×2
        >>> c = a.matmul(b)
        >>> print(c.data)
        [[19. 22.]
         [43. 50.]]

        HINTS:
        - Inner dimensions must match: (M, K) @ (K, N) = (M, N)
        - For 2D case: use np.dot(a[i, :], b[:, j]) for each output element
        - Raise ValueError with clear message if shapes incompatible
        """
        if not isinstance(other, Tensor):
            raise TypeError("Both elements must be tensors")
        if other.data.ndim == 0:
            return self * other
        if other.data.ndim == 1:
            return np.matmul(self.data, other.data)
        if not self.shape[-1] == other.shape[-2]:
            raise ValueError(f"Inner dimensions must match. {self.shape[-1]} ≠ {other.shape[-2]}")
        if self.data.shape == (2,2):
            lst = []
            for i in range(2):
                for j in range(2):
                    result = np.dot(self.data[i,:],b.data[:,j])
                    lst.append(result)
            return Tensor(np.array(lst).reshape(2,2))
        return np.matmul(self.data, other.data)


In [161]:
#################### TESTS
matrix = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3
vector = Tensor([1, 2, 3])  # 3×1 (conceptually)
result = matrix.matmul(vector)
result
# Expected: [1×1+2×2+3×3, 4×1+5×2+6×3] = [14, 32]
# expected = np.array([14, 32], dtype=np.float32)
# assert np.array_equal(result.data, expected)
# matrix = Tensor([[1, 2], [3, 4]])
# vector = Tensor([10, 20])
# result = matrix + vector
# expected = np.array([[11, 22], [13, 24]], dtype=np.float32)
# assert np.array_equal(result.data, expected)
# result
# matrix.data.ndim
# a = Tensor(2)
# a.data.ndim
# matrix.matmul(a)
#################### TESTS

array([14., 32.], dtype=float32)

# Unit tests

In [162]:
def test_unit_matrix_multiplication():
    """🧪 Test matrix multiplication operations."""
    print("🧪 Unit Test: Matrix Multiplication...")

    # Test 2×2 matrix multiplication (basic case)
    a = Tensor([[1, 2], [3, 4]])  # 2×2
    b = Tensor([[5, 6], [7, 8]])  # 2×2
    result = a.matmul(b)
    # Expected: [[1×5+2×7, 1×6+2×8], [3×5+4×7, 3×6+4×8]] = [[19, 22], [43, 50]]
    expected = np.array([[19, 22], [43, 50]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test rectangular matrices (common in neural networks)
    c = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3 (like batch_size=2, features=3)
    d = Tensor([[7, 8], [9, 10], [11, 12]])  # 3×2 (like features=3, outputs=2)
    result = c.matmul(d)
    # Expected: [[1×7+2×9+3×11, 1×8+2×10+3×12], [4×7+5×9+6×11, 4×8+5×10+6×12]]
    expected = np.array([[58, 64], [139, 154]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test matrix-vector multiplication (common in forward pass)
    matrix = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3
    vector = Tensor([1, 2, 3])  # 3×1 (conceptually)
    result = matrix.matmul(vector)
    # Expected: [1×1+2×2+3×3, 4×1+5×2+6×3] = [14, 32]
    expected = np.array([14, 32], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test shape validation - should raise clear error
    try:
        incompatible_a = Tensor([[1, 2]])     # 1×2
        incompatible_b = Tensor([[1], [2], [3]])  # 3×1
        incompatible_a.matmul(incompatible_b)  # 1×2 @ 3×1 should fail (2 ≠ 3)
        assert False, "Should have raised ValueError for incompatible shapes"
    except ValueError as e:
        assert "Inner dimensions must match" in str(e)
        assert "2 ≠ 3" in str(e)  # Should show specific dimensions

    print("✅ Matrix multiplication works correctly!")

if __name__ == "__main__":
    test_unit_matrix_multiplication()

🧪 Unit Test: Matrix Multiplication...
✅ Matrix multiplication works correctly!


In [143]:
def test_unit_arithmetic_operations():
    """🧪 Test arithmetic operations with broadcasting."""
    print("🧪 Unit Test: Arithmetic Operations...")

    # Test tensor + tensor
    a = Tensor([1, 2, 3])
    b = Tensor([4, 5, 6])
    result = a + b
    assert np.array_equal(result.data, np.array([5, 7, 9], dtype=np.float32))

    # Test tensor + scalar (very common in ML)
    result = a + 10
    assert np.array_equal(result.data, np.array([11, 12, 13], dtype=np.float32))

    # Test broadcasting with different shapes (matrix + vector)
    matrix = Tensor([[1, 2], [3, 4]])
    vector = Tensor([10, 20])
    result = matrix + vector
    expected = np.array([[11, 22], [13, 24]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test subtraction (data centering)
    result = b - a
    assert np.array_equal(result.data, np.array([3, 3, 3], dtype=np.float32))

    # Test multiplication (scaling)
    result = a * 2
    assert np.array_equal(result.data, np.array([2, 4, 6], dtype=np.float32))

    # Test division (normalization)
    result = b / 2
    assert np.array_equal(result.data, np.array([2.0, 2.5, 3.0], dtype=np.float32))

    # Test chaining operations (common in ML pipelines)
    normalized = (a - 2) / 2  # Center and scale
    expected = np.array([-0.5, 0.0, 0.5], dtype=np.float32)
    assert np.allclose(normalized.data, expected)

    print("✅ Arithmetic operations work correctly!")

if __name__ == "__main__":
    test_unit_arithmetic_operations()

🧪 Unit Test: Arithmetic Operations...
✅ Arithmetic operations work correctly!


# Experiments

In [ ]:
c = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3 (like batch_size=2, features=3)
d = Tensor([[7, 8], [9, 10], [11, 12]])  # 3×2 (like features=3, outputs=2)
result = c.matmul(d)


In [ ]:
A = np.array([[2,2], [2,2]])
for i in A:
    print(i)

In [ ]:
a = Tensor([[1, 2], [3, 4]])  # 2×2
b = Tensor([[5, 6], [7, 8]])  # 2×2

In [ ]:
lst = []
d = np.zeros([2,2])
for i in range(2):
    for j in range(2):
        print(a.data[i,:],b.data[:,j])
        print(np.dot(a.data[i,:],b.data[:,j]))
        lst.append(np.dot(a.data[i,:],b.data[:,j]))
lst = np.array(lst).reshape(2,2)
lst
# for i in a.data:
#     for j in b.data:
#         print(a.data[i,:])

In [ ]:
a.data.ndim

In [ ]:
c = Tensor([2,3,4])
c.shape

In [ ]:
matrix = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3
vector = Tensor([1, 2, 3])  # 3×1 (conceptually)
result = matrix.matmul(vector)
result

### Method 1: Using `arr.ndim`

In [ ]:
import numpy as np

# Example 1: A 1D array
array_1d = np.array([1, 2, 3, 4, 5])
print(f"Array 1: {array_1d}\n  ndim: {array_1d.ndim}")
if array_1d.ndim == 1:
    print("  This is a 1D array.\n")
else:
    print("  This is NOT a 1D array.\n")

# Example 2: A 2D array
array_2d = np.array([[1, 2], [3, 4]])
print(f"Array 2:\n{array_2d}\n  ndim: {array_2d.ndim}")
if array_2d.ndim == 1:
    print("  This is a 1D array.\n")
else:
    print("  This is NOT a 1D array.\n")

# Example 3: A 0D array (scalar)
array_0d = np.array(10)
print(f"Array 3: {array_0d}\n  ndim: {array_0d.ndim}")
if array_0d.ndim == 1:
    print("  This is a 1D array.\n")
else:
    print("  This is NOT a 1D array.\n")

### Method 2: Using `arr.shape`

In [ ]:
import numpy as np

# Example 1: A 1D array
array_1d = np.array([1, 2, 3, 4, 5])
print(f"Array 1: {array_1d}\n  shape: {array_1d.shape}")
if len(array_1d.shape) == 1:
    print("  This is a 1D array.\n")
else:
    print("  This is NOT a 1D array.\n")

# Example 2: A 2D array
array_2d = np.array([[1, 2], [3, 4]])
print(f"Array 2:\n{array_2d}\n  shape: {array_2d.shape}")
if len(array_2d.shape) == 1:
    print("  This is a 1D array.\n")
else:
    print("  This is NOT a 1D array.\n")

# Example 3: A 0D array (scalar)
array_0d = np.array(10)
print(f"Array 3: {array_0d}\n  shape: {array_0d.shape}")
if len(array_0d.shape) == 1:
    print("  This is a 1D array.\n")
else:
    print("  This is NOT a 1D array.\n")

In [ ]:
a * c

In [ ]:
a.size

In [102]:
import numpy as np

# Create a 2D NumPy array (3 rows, 4 columns)
my_2d_array = np.array([[1, 2, 3, 4],
                        [5, 6, 7, 8],
                        [9, 10, 11, 12]])

print("2D Array:\n", my_2d_array)
print("Shape:", my_2d_array.shape) # (rows, columns)
print("Number of dimensions (ndim):", my_2d_array.ndim)

2D Array:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
Shape: (3, 4)
Number of dimensions (ndim): 2
